# Apply Triggering and Evaluate

In [1]:
import os
os.chdir(f'{os.getcwd()}/../../')
WORKING_DIR = './evaluation/trigger'

In [2]:
import imageio
import numpy as np
import pandas as pd
from tqdm import tqdm
import utils3d as u3d
from datasets.xihe.loader import XiheTestDataset
from datasets.neural_illumination import NeuralIlluminationList

In [3]:
def get_neighbors(n_anchors, n_neighbors):
    sphere_anchors = u3d.fibonacci_sphere(n_anchors)
    neighbors = []
    for i, anchor in enumerate(sphere_anchors):
        neighbors.append([])

        for j, v in enumerate(sphere_anchors):
            # if i == j:
            #     continue

            neighbors[i].append({'index': j, 'angel': np.dot(anchor, v)})

        neighbors[i] = sorted(
            neighbors[i],
            key=lambda x: x['angel'],
            reverse=True)[:n_neighbors]
        neighbors[i] = [v['index'] for v in neighbors[i]]

    neighbors = np.array(neighbors, dtype=np.int)

    return neighbors

In [4]:
sv_pairs = np.load(f'{WORKING_DIR}/data/sv_pairs.npy')
nsv_pairs = np.load(f'{WORKING_DIR}/data/nsv_pairs.npy')

pairs = np.concatenate((sv_pairs, nsv_pairs), axis=0)
test_dataset = XiheTestDataset(n_points=1280)
ni_test = NeuralIlluminationList('test')

neighbors_4 = get_neighbors(1280, 4)
neighbors_9 = get_neighbors(1280, 9)
neighbors_16 = get_neighbors(1280, 16)

In [5]:
df = pd.DataFrame(columns=['data_idx', 'trigger_method', 'value'])

In [6]:
def image_trigger(m, n):
    root = '/mnt/IRONWOLF1/yiqinzhao/RawDatasets/Matterport3D/v1/scans/'
    obs_m = imageio.imread(f'{root}/{m["scene_id"]}/undistorted_color_images/{m["observation_image"]}.jpg') / 255
    obs_n = imageio.imread(f'{root}/{n["scene_id"]}/undistorted_color_images/{n["observation_image"]}.jpg') / 255
    
    return np.mean((obs_m - obs_n) ** 2)

In [7]:
def sphere_trigger(m, n):
    return np.mean((m - n) ** 2)

In [8]:
def xihe_trigger(clr_m, clr_n, neighbors):
    clr_m_window = clr_m[neighbors, :]
    clr_n_window = clr_n[neighbors, :]
    
    mse = np.mean((clr_m_window - clr_n_window) ** 2, axis=-1)
    mse_window = np.mean(mse, axis=-1)
    max_mse = np.max(mse_window)
    
    return max_mse

In [9]:
i = 0
p = pairs[i]
m, n = p

x_m = test_dataset[m][0][1].numpy().transpose()
x_n = test_dataset[n][0][1].numpy().transpose()

In [14]:
x_m[neighbors_4].shape

(1280, 4, 3)

In [16]:
for i in tqdm(range(len(pairs))):
    p = pairs[i]
    m, n = p
    
    x_m = test_dataset[m][0][1].numpy().transpose()
    x_n = test_dataset[n][0][1].numpy().transpose()
    
    df.loc[len(df)] = [i, 'image', image_trigger(ni_test[m], ni_test[n])]
    df.loc[len(df)] = [i, 'xihe_1', sphere_trigger(x_m, x_n)]
    df.loc[len(df)] = [i, 'xihe_4', xihe_trigger(x_m, x_n, neighbors_4)]
    df.loc[len(df)] = [i, 'xihe_9', xihe_trigger(x_m, x_n, neighbors_9)]
    df.loc[len(df)] = [i, 'xihe_16', xihe_trigger(x_m, x_n, neighbors_16)]

100%|██████████| 1754/1754 [02:38<00:00, 11.07it/s]


In [17]:
df.to_csv(f'{WORKING_DIR}/results/trigger_values.csv')